In [ ]:
import os
import pandas as pd
import plotly.offline as pyo
from czitools import misc
from ipyfilechooser import FileChooser
from IPython.display import display
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
#from matplotlib import cm, use
#import matplotlib.colors as colors
#import matplotlib.patches as mpatches
import plotly.graph_objects as go
import numpy as np

pyo.init_notebook_mode()

In [ ]:
defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
fc = FileChooser(defaultdir)
fc.filter_pattern = '*.czi'
display(fc)

In [ ]:
def scatterplot_mpl(planetable,
                    s=0, t=0, z=0, c=0,
                    msz2d=35,
                    normz=True,
                    fig1savename='zsurface2d.png',
                    fig2savename='zsurface3d.png',
                    show3d=False,
                    msz3d=20):

    # extract XYZ positions
    try:
        xpos = planetable['X[micron]']
        ypos = planetable['Y[micron]']
        zpos = planetable['Z[micron]']
    except KeyError as e:
        xpos = planetable['X [micron]']
        ypos = planetable['Y [micron]']
        zpos = planetable['Z [micron]']

    # normalize z-data by substracting the minimum value
    if normz:
        zpos = zpos - zpos.min()

    # create a name for the figure
    figtitle = 'XYZ-Positions:  S=' + str(s) + ' T=' + str(t) + ' Z=' + str(z) + ' CH=' + str(c)

    # try to find a "good" aspect ratio for the figures
    dx = xpos.max() - xpos.min()
    dy = ypos.max() - ypos.min()
    fsy = 8
    fsx = np.ceil(fsy * dx / dy).astype(np.int)

    # create figure
    fig1, ax1 = plt.subplots(1, 1, figsize=(fsx + 1, fsy))

    # invert the Y-axis --> O,O = Top-Left
    ax1.invert_yaxis()

    # configure the axis
    ax1.set_title(figtitle)
    ax1.set_xlabel('Stage X-Axis [micron]', fontsize=12, fontweight='normal')
    ax1.set_ylabel('Stage Y-Axis [micron]', fontsize=12, fontweight='normal')
    ax1.grid(True)
    ax1.set_aspect('equal', 'box')

    # plot data and label the colorbar
    sc1 = ax1.scatter(xpos, ypos,
                      marker='s',
                      c=zpos,
                      s=msz2d,
                      facecolor=cm.coolwarm,
                      edgecolor='black')

    # add the colorbar on the right-hand side
    cb1 = plt.colorbar(sc1,
                       fraction=0.046,
                       shrink=0.8,
                       pad=0.04)

    # add a label
    if normz:
        cb1.set_label('Z-Offset [micron]',
                      labelpad=20,
                      fontsize=12,
                      fontweight='normal')
    if not normz:
        cb1.set_label('Z-Position [micron]',
                      labelpad=20,
                      fontsize=12,
                      fontweight='normal')

    # save figure as PNG
    fig1.savefig(fig1savename, dpi=100)
    print('Saved: ', fig1savename)

    # 3D plot of surface
    fig2 = plt.figure(figsize=(fsx + 1, fsy))
    ax2 = fig2.add_subplot(111, projection='3d')

    # invert the Y-axis --> O,O = Top-Left
    ax2.invert_yaxis()

    # define the labels
    ax2.set_xlabel('Stage X-Axis [micron]',
                   fontsize=12,
                   fontweight='normal')
    ax2.set_ylabel('Stage Y-Axis [micron]',
                   fontsize=12,
                   fontweight='normal')
    ax2.set_title(figtitle)

    # plot data and label the colorbar
    sc2 = ax2.scatter(xpos, ypos, zpos,
                      marker='.',
                      s=msz3d,
                      c=zpos,
                      facecolor=cm.coolwarm,
                      depthshade=False)

    # add colorbar to the 3d plot
    cb2 = plt.colorbar(sc2, shrink=0.8)
    # add a label
    if normz:
        cb2.set_label('Z-Offset [micron]',
                      labelpad=20,
                      fontsize=12,
                      fontweight='normal')
    if not normz:
        cb2.set_label('Z-Position [micron]',
                      labelpad=20,
                      fontsize=12,
                      fontweight='normal')

    # save figure as PNG
    fig2.savefig(fig2savename, dpi=100)
    print('Saved: ', fig2savename)

    return fig1, fig2


In [ ]:
def scatterplot_plotly(planetable,
                       s=0, t=0, z=0, c=0,
                       msz2d=35,
                       normz=True,
                       fig1savename='zsurface2d.html',
                       fig2savename='zsurface3d.html',
                       msz3d=20):

    # extract XYZ position for the selected channel
    try:
        xpos = planetable['X[micron]']
        ypos = planetable['Y[micron]']
        zpos = planetable['Z[micron]']
    except KeyError as e:
        xpos = planetable['X [micron]']
        ypos = planetable['Y [micron]']
        zpos = planetable['Z [micron]']

    # normalize z-data by substracting the minimum value
    if normz:
        zpos = zpos - zpos.min()
        scalebar_title = 'Z-Offset [micron]'
    if not normz:
        scalebar_title = 'Z-Position [micron]'

    # create a name for the figure
    figtitle = 'XYZ-Positions:  S=' + str(s) + ' T=' + str(t) + ' Z=' + str(z) + ' CH=' + str(c)

    fig1 = go.Figure(
        data=go.Scatter(
            x=xpos,
            y=ypos,
            mode='markers',
            text=np.round(zpos, 1),
            marker_symbol='square',
            marker_size=msz2d,
            marker=dict(
                color=zpos,
                colorscale='Viridis',
                line_width=2,
                showscale=True,
                colorbar=dict(thickness=10,
                              title=dict(
                                  text=scalebar_title,
                                  side='right'))
            )
        )
    )

    fig1.update_xaxes(showgrid=True, zeroline=True, automargin=True)
    fig1.update_yaxes(showgrid=True, zeroline=True, automargin=True)
    fig1['layout']['yaxis']['autorange'] = "reversed"
    fig1.update_layout(title=figtitle,
                       xaxis_title="StageX Position [micron]",
                       yaxis_title="StageY Position [micron]",
                       font=dict(size=16,
                                 color='Black')
                       )

    # save the figure
    fig1.write_html(fig1savename)
    print('Saved: ', fig1savename)

    fig2 = go.Figure(data=[go.Scatter3d(
        x=xpos,
        y=ypos,
        z=zpos,
        mode='markers',
        marker=dict(
            size=msz3d,
            color=zpos,
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(thickness=10,
                          title=dict(
                              text=scalebar_title,
                              side='right')
                          )
        )
    )])

    fig2.update_xaxes(showgrid=True, zeroline=True, automargin=True)
    fig2.update_yaxes(showgrid=True, zeroline=True, automargin=True)
    fig2['layout']['yaxis']['autorange'] = "reversed"
    fig2.update_layout(title=figtitle,
                       xaxis_title="StageX Position [micron]",
                       yaxis_title="StageY Position [micron]",
                       font=dict(size=16,
                                 color='Black')
                       )

    # save the figure
    fig2.write_html(fig2savename)
    print('Saved: ', fig2savename)

    return fig1, fig2

In [ ]:
isczi = False
iscsv = False

# check if the input is a csv or czi file
if fc.selected.lower().endswith('.czi'):
    isczi = True
if fc.selected.lower().endswith('.csv'):
    iscsv = True

In [ ]:
# separator of CSV file
separator=','

# read the data from CSV file
if iscsv:
    planetable = pd.read_csv(fc.selected, sep=separator)
if isczi:
    # read the data from CSV file
    planetable, csvfile = misc.get_planetable(fc.selected)


planetable[:3]

In [ ]:
# specify the output format - 'mpl' or 'html'
plot_type='html'

# define plot type
if plot_type == 'mpl':
    saveformat = 'png'
if plot_type == 'html':
    saveformat = 'html'
    
# define name for figure to be saved
filename = os.path.basename(fc.selected)
fig1savename = os.path.splitext(fc.selected)[0] + '_XYZ-Pos' + '.' + saveformat
fig2savename = os.path.splitext(fc.selected)[0] + '_XYZ-Pos3D' + '.' + saveformat

In [ ]:
# filter the planetable for S, T, Z and C entries
planetable_filtered = misc.filter_planetable(planetable, s=0, t=0, z=0, c=0)
planetable_filtered[:5]

In [ ]:
if plot_type == 'mpl':
    # display the XYZ positions using matplotlib
    fig1, fig2 = scatterplot_mpl(planetable_filtered,
                                 s=0, t=0, z=0, c=0,
                                 msz2d=100,
                                 normz=True,
                                 fig1savename=fig1savename,
                                 fig2savename=fig2savename,
                                 msz3d=10)

if plot_type == 'html':
    # display the XYZ positions using plotly
    fig1, fig2 = scatterplot_plotly(planetable_filtered,
                                    s=0, t=0, z=0, c=0,
                                    msz2d=20,
                                    normz=True,
                                    fig1savename=fig1savename,
                                    fig2savename=fig2savename,
                                    msz3d=10)

    pyo.iplot(fig1)
    pyo.iplot(fig2)

In [ ]:
# write the planetable to a csv
csvfile = misc.save_planetable(planetable, fc.selected,
                                separator=separator,
                                index=False)

print('Write to CSV File : ', csvfile)